In [ ]:
import pandas as pd
import statsmodels.api as sm

ventas_df = pd.read_csv('/workspace/ventas_peliculas.csv')
promocion_df = pd.read_csv('/workspace/promocion_peliculas.csv')

merged_df = ventas_df.merge(promocion_df, on=['fecha', 'titulo'], how='left')

print("Valores nulos en el dataset:")
print(merged_df.isna().sum())

merged_df.fillna(0, inplace=True)

print("Resumen de ventas de boletos semanales:")
print(merged_df[['fecha', 'ventas_boletos']].groupby('fecha').sum())

print("Ventas promedio con y sin promociones:")
print(merged_df.groupby(merged_df['eventos_promocionales'] > 0)['ventas_boletos'].mean())

q1 = merged_df['ventas_boletos'].quantile(0.25)
q3 = merged_df['ventas_boletos'].quantile(0.75)
iqr = q3 - q1
outliers = merged_df[(merged_df['ventas_boletos'] < (q1 - 1.5 * iqr)) | (merged_df['ventas_boletos'] > (q3 + 1.5 * iqr))]
print("Valores extremos detectados en ventas de boletos:")
print(outliers)

correlation_matrix = merged_df[['ventas_boletos', 'calificacion', 'eventos_promocionales', 'participacion_promocion']].corr()
print("Matriz de correlación:")
print(correlation_matrix)

merged_df.plot(x='fecha', y='ventas_boletos', title='Ventas de boletos semanales', grid=True)
merged_df.groupby('eventos_promocionales')['ventas_boletos'].mean().plot(kind='bar', title='Ventas promedio con y sin promociones', grid=True)
merged_df[['ventas_boletos']].boxplot()
correlation_matrix.style.background_gradient(cmap='coolwarm')

X = merged_df[['calificacion', 'eventos_promocionales', 'participacion_promocion']]
X = sm.add_constant(X)
Y = merged_df['ventas_boletos']

model = sm.OLS(Y, X).fit()
print(model.summary())

merged_df['ventas_predichas'] = model.predict(X)
print("Comparación entre ventas reales y predichas:")
print(merged_df[['ventas_boletos', 'ventas_predichas']].head())

merged_df[['ventas_boletos', 'ventas_predichas']].plot(title='Ventas reales vs. predichas', grid=True)